In [1]:
import torch
import torch.nn as nn
import pickle as pk
import numpy as np
from model import LocalPredictor
import random

In [2]:
data = dict({})
for d in range(1, 29):
    filename = '/data/fan/UsersInTokyoProcessed/201210{:02d}_interp_samplingrate01.pk'.format(d)
    print(filename)
    with open(filename, 'rb') as f:
        data[d] = pk.load(f)

/data/fan/UsersInTokyoProcessed/20121001_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121002_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121003_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121004_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121005_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121006_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121007_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121008_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121009_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121010_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121011_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121012_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121013_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121014_interp_samplingrate01.pk
/data/fan/UsersInTokyoProcessed/20121015_interp_samplingrate01.pk
/data/fan/

In [3]:
data_valid = dict({})
for d in range(1, 29):
    filename = '/data/fan/UsersInTokyoProcessed/201210{:02d}_interp.pk'.format(d)
    print(filename)
    with open(filename, 'rb') as f:
        data_valid[d] = pk.load(f)

/data/fan/UsersInTokyoProcessed/20121001_interp.pk
/data/fan/UsersInTokyoProcessed/20121002_interp.pk
/data/fan/UsersInTokyoProcessed/20121003_interp.pk
/data/fan/UsersInTokyoProcessed/20121004_interp.pk
/data/fan/UsersInTokyoProcessed/20121005_interp.pk
/data/fan/UsersInTokyoProcessed/20121006_interp.pk
/data/fan/UsersInTokyoProcessed/20121007_interp.pk
/data/fan/UsersInTokyoProcessed/20121008_interp.pk
/data/fan/UsersInTokyoProcessed/20121009_interp.pk
/data/fan/UsersInTokyoProcessed/20121010_interp.pk
/data/fan/UsersInTokyoProcessed/20121011_interp.pk
/data/fan/UsersInTokyoProcessed/20121012_interp.pk
/data/fan/UsersInTokyoProcessed/20121013_interp.pk
/data/fan/UsersInTokyoProcessed/20121014_interp.pk
/data/fan/UsersInTokyoProcessed/20121015_interp.pk
/data/fan/UsersInTokyoProcessed/20121016_interp.pk
/data/fan/UsersInTokyoProcessed/20121017_interp.pk
/data/fan/UsersInTokyoProcessed/20121018_interp.pk
/data/fan/UsersInTokyoProcessed/20121019_interp.pk
/data/fan/UsersInTokyoProcessed

In [4]:
uid_sets = dict({})
for d in data:
    uid_sets[d] = set(list(data[d].keys()))

In [5]:
uid_doc_set = set([])
for d in range(1, 15):
    uid_doc_set = uid_doc_set | uid_sets[d]

In [6]:
data_doc = dict({})
data_doc_test = dict({})
data_qry_train = dict({})
data_qry_test = dict({})

for d in range(1, 15):
    for uid in data[d]:
        if uid not in data_doc:
            data_doc[uid] = [data[d][uid]]
            data_doc_test[uid] = [data_valid[d][uid]]
        else:
            data_doc[uid].append(data[d][uid])
            data_doc_test[uid].append(data_valid[d][uid])

for d in range(15, 22):
    for uid in data[d]:
        if uid not in data_qry_train:
            data_qry_train[uid] = [data[d][uid]]
        else:
            data_qry_train[uid].append(data[d][uid])
            
for d in range(22, 29):
    for uid in data[d]:
        if uid not in data_qry_test:
            data_qry_test[uid] = [data_valid[d][uid]]
        else:
            data_qry_test[uid].append(data_valid[d][uid])

In [7]:
for uid in data_doc:
    data_doc[uid] = torch.LongTensor(data_doc[uid]).cuda(0)
    
for uid in data_doc_test:
    data_doc_test[uid] = torch.LongTensor(data_doc_test[uid]).cuda(0)

for uid in data_qry_train:
    data_qry_train[uid] = torch.LongTensor(data_qry_train[uid]).cuda(0)
    
for uid in data_qry_test:
    data_qry_test[uid] = torch.LongTensor(data_qry_test[uid]).cuda(0)

In [8]:
num_locs = 1600
loc_embedding_dim = 128
T = 96
num_time = T
time_embedding_dim = 32
hidden_dim = 256
latent_dim = 256
n_layers = 2

In [9]:
dT = 4

In [10]:
training_loss = dict({})
validation_loss = dict({})

In [12]:
batch_size = 128
user_list_train = list(data_qry_train.keys())


local_predictor = LocalPredictor(num_locs, loc_embedding_dim, num_time, time_embedding_dim, hidden_dim, latent_dim, n_layers).cuda(0)

optimizer = torch.optim.RMSprop(local_predictor.parameters(), lr=1e-3)

for epoch in range(1, 11):
    optimizer.zero_grad()

    random.shuffle(user_list_train)

    avg_loss = 0.0
    cnt = 0

    for uid in user_list_train:
        t = np.random.randint(T - 2 * dT + 1)
        x_loc_qry = data_qry_train[uid][:, t: t + dT]
        x_t_qry = torch.zeros_like(x_loc_qry) + t
        y = data_qry_train[uid][:, t + 2 * dT - 1]

        if uid not in data_doc:
            loss = local_predictor(x_loc_qry, x_t_qry, None, None, y)
        else:
            x_loc_doc = []
            x_t_doc = []
            for j in range(0, 1):
                if t + j >= 0 and t + 2 * dT + j <= T:
                    tmp = data_doc[uid][:, t + j: t + 2 * dT + j]
                    x_loc_doc.append(tmp)
                    x_t_doc.append(torch.zeros_like(tmp) + t + j)
            loss = local_predictor(x_loc_qry, x_t_qry, torch.cat(x_loc_doc, dim=0), torch.cat(x_t_doc, dim=0), y)

        loss.backward()
        cnt += data_qry_train[uid].shape[0]
        avg_loss += loss.item()

        if cnt % batch_size == 0:
            optimizer.step()
            optimizer.zero_grad()

    print('Training Loss = {:.4f}'.format(avg_loss / cnt))

    # testing
    cnt = 0
    avg_loss = 0.0

    with torch.no_grad():
        for uid in data_qry_test:
            t = np.random.randint(T - 2 * dT + 1)
            x_loc_qry = data_qry_test[uid][:, t: t + dT]
            x_t_qry = torch.zeros_like(x_loc_qry) + t
            y = data_qry_test[uid][:, t + 2 * dT - 1]

            if uid not in data_doc:
                loss = local_predictor(x_loc_qry, x_t_qry, None, None, y)
            else:
                x_loc_doc = []
                x_t_doc = []
                for j in range(0, 1):
                    if t + j >= 0 and t + 2 * dT + j <= T:
                        tmp = data_doc_test[uid][:, t + j: t + 2 * dT + j]
                        x_loc_doc.append(tmp)
                        x_t_doc.append(torch.zeros_like(tmp) + t + j)
                loss = local_predictor(x_loc_qry, x_t_qry, torch.cat(x_loc_doc, dim=0), torch.cat(x_t_doc, dim=0), y)

            cnt += data_qry_test[uid].shape[0]
            avg_loss += loss.item()

    print('Validation Loss = {:.4f}'.format(avg_loss / cnt))

Training Loss = 1.1306
Validation Loss = 2.3141
Training Loss = 0.8597
Validation Loss = 2.1701
Training Loss = 0.8340
Validation Loss = 2.0689
Training Loss = 0.8119
Validation Loss = 2.0313
Training Loss = 0.8113
Validation Loss = 2.0027
Training Loss = 0.8005
Validation Loss = 2.0105
Training Loss = 0.7997
Validation Loss = 1.9648
Training Loss = 0.7953
Validation Loss = 1.9449
Training Loss = 0.7999
Validation Loss = 1.9537
Training Loss = 0.7971
Validation Loss = 1.9721


In [21]:
import numpy as np

In [29]:
list(local_predictor.parameters())[0].detach().cpu().numpy().shape

(1600, 128)

In [36]:
list(local_predictor.parameters())[0].detach().cpu().numpy().astype(np.float16)

array([[ 0.7837  , -0.8535  ,  0.7783  , ...,  0.9575  , -1.404   ,
        -2.242   ],
       [-1.394   ,  1.734   , -1.588   , ..., -0.657   ,  0.3118  ,
         0.01277 ],
       [ 0.11224 , -0.0396  , -0.595   , ...,  0.103   ,  0.3794  ,
         0.00515 ],
       ...,
       [-0.1948  , -0.05878 ,  1.078   , ...,  0.4     ,  0.5933  ,
        -0.284   ],
       [ 0.0762  ,  0.1871  ,  0.2025  , ...,  0.3093  , -0.6455  ,
         0.7437  ],
       [ 0.7876  ,  0.2144  , -0.3123  , ...,  0.011314, -0.281   ,
        -0.3767  ]], dtype=float16)

In [ ]:
parameters_before_comp = []
parameters_after_comp = []
for para in local_predictor.parameters():
    para = para.detach().cpu().numpy()
    parameters_before_comp.append(para)
    if len(para.shape) > 1: